In [ ]:
from Bio import GenBank
import re as re
from pathlib import Path

Constants

In [ ]:
nb_of_segments = NUMBER_OF_SEGMENTS -> int
nt = PERMISSIBLE_OF_NUCLEOTIDE -> int         
nb_segments = {
    '1':  QUANTITY OF NUCLEOTIDES IN SEGMENT 1 -> int ,
    '2':  QUANTITY OF NUCLEOTIDES IN SEGMENT 2  -> int ,
    '3':  QUANTITY OF NUCLEOTIDES IN SEGMENT 3 -> int ,
    '4':  QUANTITY OF NUCLEOTIDES IN SEGMENT 4 -> int ,
    '5':  QUANTITY OF NUCLEOTIDES IN SEGMENT 5 -> int ,
    '6':  QUANTITY OF NUCLEOTIDES IN SEGMENT 6 -> int ,
    '7':  QUANTITY OF NUCLEOTIDES IN SEGMENT 7 -> int ,
    '8':  QUANTITY OF NUCLEOTIDES IN SEGMENT 8 -> int ,
                }

Functions that we used

In [ ]:
#in some cases, segment names consist of letters, we need to change them to numbers
def change_letters_to_numbers(organisms: dict) -> dict:
    for seg_seq in organisms.values():
            not_allowed = []
            if 'M' in seg_seq.keys() and 'L' in seg_seq.keys() and 'S' in seg_seq.keys():
                    seg_seq['1']=seg_seq['S']
                    seg_seq.pop('S')
                    seg_seq['2']=seg_seq['M']
                    seg_seq.pop('M')
                    seg_seq['3']=seg_seq['L']
                    seg_seq.pop('L')
            elif 'S' in seg_seq.keys() and 'L' in seg_seq.keys() and 'M' not in seg_seq.keys():
                    seg_seq['1']=seg_seq['S']
                    seg_seq.pop('S')
                    seg_seq['2']=seg_seq['L']
                    seg_seq.pop('L')
            not_allowed = list(filter(None, "".join(re.split(r'\d', "/".join(seg_seq.keys()))).replace('"',"").split("/")))
            
            if len(not_allowed) != 0:
                for wrong_name in not_allowed:
                      seg_seq.pop(wrong_name)
           
    return organisms
#we have to choose the longest segment

def choose_the_proteint_coding_sequence(lib: dict) -> dict):
    for name, content in lib.items():
        for num, data in content.items():
            #print(data)
            if len(data) > 1:
                
                
                good = []
                for item in data[1]:
                    if "(" in item:
                        good.extend(item.split("(")[1][:-1].split(","))
                    else:
                        good.append(item)
                max = 0
                best = ""
                #print(good)
                for item in good:
                    t = item.split("..")
                    #print(item)
                    i = int(t[0].strip("<>"))
                    j = int(t[1].strip("<>"))
                    if j - i > max:
                        max = j-i
                        best = item
                
                data[1] = best.replace(">","").replace("<","")
                
            else:
                data = []
    
    return lib
                

#sometimes we should cut nucleotides before first start-codon and after the last stop-codon
def cut_primers(organisms: dict,  nb_of_segments: dict) -> dict:

    for dictt in organisms.values():
        for number_of_segment, listt in dictt.items():

            if number_of_segment == '1':
                if len(listt) > 1:
                    cut_start = listt[-1].split("..")[0]
                    #if cut_start not in ["A", "T", "G", "C", "N"]:
                    cut_start = int(cut_start) - 1

                    dictt[number_of_segment] = listt[0][cut_start::]
               
            if int(number_of_segment.replace('"', "")) == nb_of_segments:
                if len(listt) > 1:
                    cut_end = listt[-1].split("..")[-1]
                    #if cut_end not in ["A", "T", "G", "C"]:
                    cut_end = int(cut_end)
                    dictt[number_of_segment] = listt[0][0:cut_end]

            elif number_of_segment !='1' and number_of_segment != nb_of_segments:
                  dictt[number_of_segment] = listt[0]
                     
    return organisms



def cut_primers_for_all_segments(organisms: dict) -> dict:

    for dictt in organisms.values():
        for number_of_segment, listt in dictt.items():

                if len(listt) > 1:
                    cut_start = listt[-1].split("..")[0]
                    cut_start = int(cut_start) - 1
                    cut_end = listt[-1].split("..")[-1]
                    cut_end = int(cut_end)
                    
                    dictt[number_of_segment] = listt[0][cut_start:cut_end]

                     
    return organisms


#check len of segments
def check_segments(organisms: dict, nb_segments: int, nt: int) -> tuple(dict, dict):
    seq2 = {}
    length_of_segments = {}
    for name, dictt in organisms.items():
        seq2[name] = {}  
        length_of_segments[name] = {}
        for k, v in nb_segments.items():  
             if k in dictt.keys():
                length_of_segments[name][k] = len(dictt[k]) 
                if (v - nt <= len(dictt[k]) <= v + nt):
                             seq2[name][k]=dictt[k]
                             

    return seq2, length_of_segments                 

#here we bring to the form {id:record of all segments}
def prep_delete_degenerate_nucleotides(seq2: dict, nb_of_segments: int) -> dict:
    prep_without_dn = {}
    for key, values in seq2.items():
        if len(values) == nb_of_segments:
         prep_without_dn[key] = "".join(str(dict(sorted(values.items(), key=lambda x: x[0])).values())).replace("'","").replace('dict_values([',"").replace('])',"").replace(',',"").replace(" ","")
         #prep_without_dn[key] = dict(sorted(values.items(), key=lambda x: x[0])).values()
   
    return prep_without_dn

#sometimes we should delete {id:record} with degenerate nucleotides
def delete_degenerate_nucleotides(prep_without_dn: dict) -> dict:
    without_dn={}
    dn = ['u', 'w', 'n', 's', 'm', 'k', 'r', 'y', 'b', 'd', 'h', 'v']
    for key3, value3 in prep_without_dn.items():
        y = 0
        for nt in dn:
            if nt in value3.lower():
                y = 1
        if y == 0:
            without_dn[key3]=value3
    return without_dn



App

In [ ]:
with open(path_to_your_genbank, "r") as handle:
                organisms = {}
                for record in GenBank.parse(handle):
                    not_protein = []  
                    future_id_all = [""]
                    number_of_segment = 'wrong'
                    segments = [] 
                    strings = []

                    for feature in record.features:
                            fut_id_var1 = ""
                            fut_id_var2 = ""
                            fut_id_var3 = ""
                            if feature.key == "CDS":
                                 not_protein.append(feature.location)
                                 
                            for qualifier in feature.qualifiers:  
                        
                                if qualifier.key == "/segment=":  
                                    number_of_segment = qualifier.value.replace('"',"")
                                    segments.append(number_of_segment)
                                if qualifier.key == "/strain=":  
                                     fut_id_var1 = qualifier.value
                                     future_id_all.append(fut_id_var1)
                                if qualifier.key ==  "/isolate=":
                                     fut_id_var2 = qualifier.value
                                     future_id_all.append(fut_id_var2)
                                #may be useful for some case
                                if qualifier.key == "/organism=":
                                     fut_id_var3 = qualifier.value
                                     future_id_all.append(fut_id_var3) 
                                fut_id = max(future_id_all, key = len)
                            
                            if fut_id not in organisms.keys():
                                 organisms[fut_id] = {}
                    organisms[fut_id][number_of_segment] = [record.sequence]
                    if not_protein:
                        organisms[fut_id][number_of_segment].append(not_protein)
 
            organisms = change_letters_to_numbers(organisms)
            organisms = choose_the_proteint_coding_sequence(organisms)
            #if you want only protein-coding-sequence:
                organisms = cut_primers_for_all_segments(organisms)
            #else (cut primers from first segment before start-codon and after the last stop-codon):
                organisms = cut_primers(organisms, nb_of_segments)
            organisms_right_len, length_of_segments = check_segments(organisms, nb_segments, nt)
            prep_without_dn = prep_delete_degenerate_nucleotides(organisms_right_len, nb_of_segments)
            without_dn = delete_degenerate_nucleotides(prep_without_dn)

            #create a file with mergening genome of virus
            with open("all_segments.fasta", "w") as f:
                for key1, item in  without_dn.items():
                    strings.append(">{} \n{}\n".format(key1, item))
                result1 = "".join(strings)
                f.write(result1)